In [10]:
import json
import pandas as pd
import timm
import tqdm

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


PATH = "model_test_results.json"
MODEL_SIZE_JSON = "model_size.json"

In [11]:
timm.__version__

'0.9.5'

In [12]:
csv_path = '../../pytorch-image-models/results/results-imagenet.csv'

In [13]:
df_results = pd.read_csv(csv_path)

In [14]:
def get_acc(df, regex):
    df = df.set_index('model')
    return df.filter(regex=regex, axis=0)

In [15]:
get_acc(df_results, '^convformer.*')

,top1,top1_err,top5,top5_err,param_count,img_size,crop_pct,interpolation
model,,,,,,,,
convformer_b36.sail_in22k_ft_in1k_384,87.602,12.398,98.434,1.566,99.88,384,1.0,bicubic
convformer_b36.sail_in22k_ft_in1k,86.998,13.002,98.172,1.828,99.88,224,1.0,bicubic
convformer_m36.sail_in22k_ft_in1k_384,86.892,13.108,98.116,1.884,57.05,384,1.0,bicubic
convformer_s36.sail_in22k_ft_in1k_384,86.378,13.622,97.984,2.016,40.01,384,1.0,bicubic
convformer_m36.sail_in22k_ft_in1k,86.146,13.854,97.850,2.150,57.05,224,1.0,bicubic
convformer_b36.sail_in1k_384,85.742,14.258,97.524,2.476,99.88,384,1.0,bicubic
convformer_m36.sail_in1k_384,85.580,14.420,97.542,2.458,57.05,384,1.0,bicubic
convformer_s36.sail_in22k_ft_in1k,85.414,14.586,97.568,2.432,40.01,224,1.0,bicubic
convformer_s36.sail_in1k_384,85.378,14.622,97.476,2.524,40.01,384,1.0,bicubic


## Get model sizes

In [16]:
with open(MODEL_SIZE_JSON, "r") as f:
    model_size_dict = json.load(f)
print(f'Current size of the dict: {len(model_size_dict)}')

Current size of the dict: 946


## Build full dataframe

In [21]:
def build_df(models: dict, model_size_dict: dict):
    model_rows = []
    for model_name, model_results in models.items():
        row = {}
        row["name"] = model_name
        if model_name in model_size_dict:
            row['model params (M)'] = model_size_dict[model_name]
        if "conversion_to_dag" in model_results:
            cvt = model_results["conversion_to_dag"]
            if cvt["success"]:
                row["conversion to torch-dag"] = "✅"
            else:
                row["conversion to torch-dag"] = "❌"
        else:
            row["conversion to torch-dag"] = "❓"
        
        if "block_pruning" in model_results:
            bp = model_results["block_pruning"]
            if bp["success"]:
                # row["block pruning"] = "✅"
                # if "score" in bp:
                #     row["bp % flops"] = f"{bp['score']*100:.1f}"
                # else:
                #     row["bp % flops"] = ""
                if "score" in bp:
                    row["block pruning proportion"] = bp['score']
                    if bp['score'] * 100 < 50:
                        row["block pruning"] = f"🟨"  # ⚠️
                    else:
                        row["block pruning"] = f"✅"
                else:
                    row["block pruning"] = "✅"
            else:
                row["block pruning"] = "❌"
        else:
            row["block pruning"] = "❓"
            
        if "channel_pruning" in model_results:
            cp = model_results["channel_pruning"]
            if cp["success"]:
                # row["channel pruning"] = "✅"
                # if "score" in cp:
                #     row["cp % flops"] = f"{cp['score']*100:.1f}"
                # else:
                #     row["cp % flops"] = "➖"
                if "score" in cp:
                    row["channel pruning proportion"] = cp['score']
                    if cp["score"] * 100 < 50:
                        row["channel pruning"] = f"🟨"
                    else:
                        row["channel pruning"] = f"✅"
                else:
                    row["channel pruning"] = "✅"
            else:
                row["channel pruning"] = "❌"
        else:
            row["channel pruning"] = "❓"
        
        model_rows.append(row)
    df = pd.DataFrame(model_rows)
    df = df.set_index('name')
    df = df.sort_index()
        
    return df
        

In [22]:
with open(PATH, "r") as f:
    models_ = json.load(f)

models_ = {k.split('.')[0]: v for k, v in models_.items()}
print(len(models_))

keys = set([m.split('.')[0] for m in models_])
models = {k: models_[k] for k in keys}


df = build_df(models, model_size_dict)

945


In [28]:
df.filter(regex='^convformer.*', axis=0)

,model params (M),conversion to torch-dag,block pruning,channel pruning,block pruning proportion,channel pruning proportion
name,,,,,,
convformer_b36,99.882616,✅,🟨,✅,0.423721,0.991101
convformer_m36,57.051640,✅,🟨,✅,0.424940,0.990808
convformer_s18,26.774448,✅,🟨,✅,0.257182,0.979026
convformer_s36,40.012152,✅,🟨,✅,0.325442,0.988298


In [17]:
df

,model params (M),conversion to torch-dag,block pruning,channel pruning,block pruning proportion,channel pruning proportion
name,,,,,,
bat_resnext26ts,10.731200,❌,❌,❌,NaN,NaN
beit_base_patch16_224,86.530984,✅,✅,✅,0.992993,0.661278
beit_base_patch16_384,86.744104,✅,✅,✅,0.993000,0.661283
beit_large_patch16_224,304.430568,✅,✅,✅,0.997320,0.664339
beit_large_patch16_384,304.998888,✅,✅,✅,0.997322,0.664341
beit_large_patch16_512,305.674728,✅,✅,✅,0.997323,0.664341
beitv2_base_patch16_224,86.530984,✅,✅,✅,0.992993,0.661278
beitv2_large_patch16_224,304.430568,✅,✅,✅,0.997320,0.664339
botnet26t_256,12.488672,❌,❌,❌,NaN,NaN


## Untested models

In [52]:
timm_models = list(set(timm.list_models('*')))
tested_models = list(set([e for e in list(df.index)]))

In [53]:
len(timm_models)

946

In [54]:
len(tested_models)

945

In [55]:
untested_models = set(timm_models) - set(tested_models)

In [56]:
todo = [m for m in untested_models if '' in m]
todo.sort()
todo

['efficientnet_l2']

In [57]:
convertible = len(df[df["conversion to torch-dag"] != "❌"])
block_prunable = len(df[df["block pruning"] == "✅"])
channel_prunable = len(df[df["channel pruning"] == "✅"])
total = len(df)

print(f"Stats:\nConvertible: {convertible/total*100:.1f}% ({convertible}/{total})\nBlock prunable: {block_prunable/total*100:.1f}% ({block_prunable}/{total})\nChannel prunable: {channel_prunable/total*100:.1f}% ({channel_prunable}/{total})")

Stats:
Convertible: 68.8% (650/945)
Block prunable: 37.0% (350/945)
Channel prunable: 58.6% (554/945)


In [58]:
all_models_list = list(df.index)
non_convertible_list = list(df[df["conversion to torch-dag"] == "❌"].index)
convertible_list = list(df[df["conversion to torch-dag"] == "✅"].index)
channel_prunable_list = list(df[df["channel pruning"] == "✅"].index)

In [59]:
md_saving_path = '../resources/supported_models_table.md'
md_table = df[df["conversion to torch-dag"] != "❌"].to_markdown()
with open(md_saving_path, "w+") as md:
    md.write(md_table)

In [60]:
def get_percentage_string(a: int, b: int) -> str:
    return f'{(a/b * 100):.2f}%'

In [61]:
data = [[total, get_percentage_string(total, total)], [convertible, get_percentage_string(convertible, total)], [channel_prunable, get_percentage_string(channel_prunable, total)]]
df_table = pd.DataFrame(data, columns=['num models', 'percentage'], index=['all models', 'convertible models', 'channel prunable_models'])

In [62]:
print(df_table.to_markdown())

|                         |   num models | percentage   |
|:------------------------|-------------:|:-------------|
| all models              |          945 | 100.00%      |
| convertible models      |          650 | 68.78%       |
| channel prunable_models |          554 | 58.62%       |


## Table for pruning support

In [63]:
df_table

,num models,percentage
all models,945,100.00%
convertible models,650,68.78%
channel prunable_models,554,58.62%


In [64]:
md_saving_path = '../resources/channel_pruning_supported_models_table.md'
md_table = df[df["conversion to torch-dag"] != "❌"].to_markdown()
with open(md_saving_path, "w+") as md:
    md.write(md_table)

In [65]:
df = df.drop('block pruning', axis=1)
df = df.drop('block pruning proportion', axis=1)

In [66]:
df

,model params (M),conversion to torch-dag,channel pruning proportion,channel pruning
name,,,,
bat_resnext26ts,10.731200,❌,NaN,❌
beit_base_patch16_224,86.530984,✅,0.661278,✅
beit_base_patch16_384,86.744104,✅,0.661283,✅
beit_large_patch16_224,304.430568,✅,0.664339,✅
beit_large_patch16_384,304.998888,✅,0.664341,✅
beit_large_patch16_512,305.674728,✅,0.664341,✅
beitv2_base_patch16_224,86.530984,✅,0.661278,✅
beitv2_large_patch16_224,304.430568,✅,0.664339,✅
botnet26t_256,12.488672,❌,NaN,❌


In [67]:
md_saving_path = '../resources/channel_pruning_supported_models_table.md'
md_table = df.to_markdown()
with open(md_saving_path, "w") as md:
    md.write(md_table)